# **Plots**

### GUI screen for creating plots and show stats
### recipe size distribution and frequency rank distribution plots are created with all recipes at once and one with separate meal type recipes and ingredients.

***Created by Rahul Maheshwari***

In [ ]:
#! /usr/bin/env python
#  -*- coding: utf-8 -*-
import operator
import pickle
import re
import sys
from collections import Counter
import matplotlib.pyplot as plt
import plot_support
import pandas as pd

**Populate GUI**

In [ ]:
try:
    import Tkinter as tk
except ImportError:
    import tkinter as tk

try:
    import ttk

    py3 = False
except ImportError:
    import tkinter.ttk as ttk

    py3 = True


def vp_start_gui():
    """Starting point when module is the main routine."""
    global val, w, root
    root = tk.Tk()
    plot_support.set_Tk_var()
    top = Toplevel1(root)
    plot_support.init(root, top)
    root.mainloop()


w = None


def create_Toplevel1(rt, *args, **kwargs):
    """Starting point when module is imported by another module.
       Correct form of call: 'create_Toplevel1(root, *args, **kwargs)' ."""
    global w, w_win, root
    # rt = root
    root = rt
    w = tk.Toplevel(root)
    plot_support.set_Tk_var()
    top = Toplevel1(w)
    plot_support.init(w, top, *args, **kwargs)
    return (w, top)


def destroy_Toplevel1():
    global w
    w.destroy()
    w = None


## **Class containing methods to make plots and show stats selected on the GUI screen**

In [ ]:
class Toplevel1:
    def make_plot(self):
        plot_name = str(self.TCombobox1.get())
        if plot_name == 'recipe rank dist. (all)':
            dbfile = open('recipe_size_dict_pickle', 'rb')
            recipe_size_dict = pickle.load(dbfile)
            dbfile1 = open('max_recipe_len', 'rb')
            max_recipe_len = pickle.load(dbfile1)
            plt.plot(*zip(*recipe_size_dict.items()))
            plt.xticks(range(0, max_recipe_len + 1))
            plt.xlabel('Recipe size')
            plt.ylabel("Percentage (%)")
            plt.title("Recipe size distribution (all recipes)")
            plt.show()
        if plot_name == 'freq rank dist. (all)':
            dbfile2 = open('ingredients_counter_plot_dict.pkl', 'rb')
            # source, destination
            ingredients_counter_plot_dict = pickle.load(dbfile2)
            plt.plot(*zip(*ingredients_counter_plot_dict.items()))
            plt.xlabel('Rank')
            plt.ylabel("Frequency")
            plt.xscale('log')
            plt.yscale('log')
            plt.title("Frequency-rank distribution (all recipes)")
            plt.show()
        if plot_name == 'recipe rank dist. (meal type)':
            df = pd.read_csv("recipes.csv")
            breakfast_df = df[df["Meal"] == "Breakfast"]
            lunch_df = df[df["Meal"] == "Lunch"]
            dinner_df = df[df["Meal"] == "Dinner"]

            meals_df_dict = {}
            meals_df_dict["breakfast"] = breakfast_df
            meals_df_dict["lunch"] = lunch_df
            meals_df_dict["dinner"] = dinner_df

            meals = ["breakfast", "lunch", "dinner"]
            meals_ingredients_dict = {}
            for meal in meals:
                ingredients_lists = list(meals_df_dict[meal]["Lookup Ingredients"])
                ingredients = []
                recipe_size_dict = {}
                recipe_size_dict[0] = 0
                recipe_count = len(df)
                for i, ingredient_list in enumerate(ingredients_lists):
                    temp_list = ingredient_list.strip('][').split(', ')
                    temp_list = [re.sub("'", "", t) for t in temp_list]
                    if len(temp_list) not in recipe_size_dict.keys():
                        recipe_size_dict.setdefault(len(temp_list), 0)
                    recipe_size_dict[len(temp_list)] += 1
                    for t in temp_list:
                        if t is not "":
                            ingredients.append(t.strip())

                meals_ingredients_dict[meal] = ingredients

                for key, val in recipe_size_dict.items():
                    recipe_size_dict[key] = val / recipe_count
                recipe_size_dict = dict(sorted(recipe_size_dict.items()))

                plt.plot(*zip(*recipe_size_dict.items()))
            dbfile1 = open('max_recipe_len', 'rb')
            max_recipe_len = pickle.load(dbfile1)
            plt.xticks(range(0, max_recipe_len + 1))
            plt.xlabel('Recipe size')
            plt.ylabel("Percentage (%)")
            plt.legend(meals)
            plt.title("Recipe size distribution (specific meal type)")
            plt.show()
        if plot_name == 'freq rank dist. (meal type)':
            df = pd.read_csv("recipes.csv")
            breakfast_df = df[df["Meal"] == "Breakfast"]
            lunch_df = df[df["Meal"] == "Lunch"]
            dinner_df = df[df["Meal"] == "Dinner"]

            meals_df_dict = {}
            meals_df_dict["breakfast"] = breakfast_df
            meals_df_dict["lunch"] = lunch_df
            meals_df_dict["dinner"] = dinner_df

            meals = ["breakfast", "lunch", "dinner"]
            meals_ingredients_dict = {}
            for meal in meals:
                ingredients_lists = list(meals_df_dict[meal]["Lookup Ingredients"])
                ingredients = []
                recipe_size_dict = {}
                recipe_size_dict[0] = 0
                for i, ingredient_list in enumerate(ingredients_lists):
                    temp_list = ingredient_list.strip('][').split(', ')
                    temp_list = [re.sub("'", "", t) for t in temp_list]
                    if len(temp_list) not in recipe_size_dict.keys():
                        recipe_size_dict.setdefault(len(temp_list), 0)
                    recipe_size_dict[len(temp_list)] += 1
                    for t in temp_list:
                        if t is not "":
                            ingredients.append(t.strip())

                meals_ingredients_dict[meal] = ingredients
            recipe_count = len(df)
            for meal in meals:
                ingredients_counter_dict = dict(Counter(meals_ingredients_dict[meal]))
                ingredients_counter_dict = dict(
                    sorted(ingredients_counter_dict.items(), key=operator.itemgetter(1), reverse=True))
                ingredients_counter_plot_dict = {i: v / recipe_count for i, (k, v) in
                                                 enumerate(ingredients_counter_dict.items(), 1)}
                plt.plot(*zip(*ingredients_counter_plot_dict.items()))

            plt.xlabel('Rank')
            plt.ylabel("Frequency")
            plt.xscale('log')
            plt.yscale('log')
            plt.legend(meals)
            plt.title("Frequency-rank distribution (specific meal type)")
            plt.show()

    def display_stat(self):
        dbfile3 = open('top_N_overall.pkl', 'rb')
        dbfile4 = open('top_N_dict.pkl', 'rb')
        top_N_overall = pickle.load(dbfile3)
        top_N_dict = pickle.load(dbfile4)
        meal_type = str(self.TCombobox2.get())
        if meal_type == 'Top ingredients Overall':
            self.top_text.configure(text="Top ingredients Overall")
            top_string = ', '.join(top_N_overall)
            self.top_text_4.configure(text=top_string)
        if meal_type == 'Top 10 Breakfast ingredients':
            self.top_text.configure(text="Top 10 Breakfast ingredients")
            top_string = ', '.join(top_N_dict["breakfast"])
            self.top_text_4.configure(text=top_string)
        if meal_type == 'Top 10 Lunch ingredients':
            self.top_text.configure(text="Top 10 Lunch ingredients")
            top_string = ', '.join(top_N_dict["lunch"])
            self.top_text_4.configure(text=top_string)
        if meal_type == 'Top 10 Dinner ingredients':
            self.top_text.configure(text="Top 10 Dinner ingredients")
            top_N = top_N_dict["dinner"]
            top_N.remove('black')
            top_N.append('black pepper')
            top_string = ', '.join(top_N)
            self.top_text_4.configure(text=top_string)

    def __init__(self, top=None):
        '''This class configures and populates the toplevel window.
           top is the toplevel containing window.'''
        _bgcolor = '#d9d9d9'  # X11 color: 'gray85'
        _fgcolor = '#000000'  # X11 color: 'black'
        _compcolor = '#d9d9d9'  # X11 color: 'gray85'
        _ana1color = '#d9d9d9'  # X11 color: 'gray85'
        _ana2color = '#ececec'  # Closest X11 color: 'gray92'
        font10 = "-family {Segoe UI} -size 14 -weight bold"
        font11 = "-family {MS PGothic} -size 14"
        font12 = "-family {Century Gothic} -size 24 -weight bold"
        font14 = "-family {Microsoft YaHei} -size 16"
        font15 = "-family {Microsoft YaHei} -size 16 -weight bold"
        self.style = ttk.Style()
        if sys.platform == "win32":
            self.style.theme_use('winnative')
        self.style.configure('.', background=_bgcolor)
        self.style.configure('.', foreground=_fgcolor)
        self.style.configure('.', font="TkDefaultFont")
        self.style.map('.', background=
        [('selected', _compcolor), ('active', _ana2color)])

        top.geometry("1920x1001+650+150")
        top.attributes("-fullscreen", True)
        top.minsize(148, 1)
        top.maxsize(1924, 1055)
        top.resizable(1, 1)
        top.title("New Toplevel")
        top.configure(background="#64ccc7")

        self.Label1 = tk.Label(top)
        self.Label1.place(relx=0.464, rely=0.12, height=46, width=111)
        self.Label1.configure(background="#64ccc7")
        self.Label1.configure(disabledforeground="#a3a3a3")
        self.Label1.configure(font=font12)
        self.Label1.configure(foreground="#ffffff")
        self.Label1.configure(text='''Plots''')

        plot_list = ['recipe rank dist. (all)', 'freq rank dist. (all)', 'recipe rank dist. (meal type)',
                     'freq rank dist. (meal type)']
        self.TCombobox1 = ttk.Combobox(top, values=plot_list, state='readonly')
        self.TCombobox1.place(relx=0.411, rely=0.31, relheight=0.036
                              , relwidth=0.170)
        self.TCombobox1.configure(font=font10)
        self.TCombobox1.configure(textvariable=plot_support.combobox)
        self.TCombobox1.configure(takefocus="")

        self.Label1_1 = tk.Label(top)
        self.Label1_1.place(relx=0.464, rely=0.257, height=46, width=111)
        self.Label1_1.configure(activebackground="#f9f9f9")
        self.Label1_1.configure(activeforeground="black")
        self.Label1_1.configure(background="#64ccc7")
        self.Label1_1.configure(disabledforeground="#a3a3a3")
        self.Label1_1.configure(font="-family {MS PGothic} -size 14 -weight bold")
        self.Label1_1.configure(foreground="#ffffff")
        self.Label1_1.configure(highlightbackground="#d9d9d9")
        self.Label1_1.configure(highlightcolor="black")
        self.Label1_1.configure(text='''Select plot''')

        self.Button1 = tk.Button(top)
        self.Button1.place(relx=0.464, rely=0.37, height=43, width=108)
        self.Button1.configure(activebackground="#ececec")
        self.Button1.configure(activeforeground="#000000")
        self.Button1.configure(background="#1d9194")
        self.Button1.configure(disabledforeground="#a3a3a3")
        self.Button1.configure(font=font10)
        self.Button1.configure(foreground="#ffffff")
        self.Button1.configure(highlightbackground="#d9d9d9")
        self.Button1.configure(highlightcolor="black")
        self.Button1.configure(pady="0")
        self.Button1.configure(text='''Plot''')
        self.Button1.configure(command=self.make_plot)

        self.Label1_2 = tk.Label(top)
        self.Label1_2.place(relx=0.464, rely=0.559, height=46, width=111)
        self.Label1_2.configure(activebackground="#f9f9f9")
        self.Label1_2.configure(activeforeground="black")
        self.Label1_2.configure(background="#64ccc7")
        self.Label1_2.configure(disabledforeground="#a3a3a3")
        self.Label1_2.configure(font=font12)
        self.Label1_2.configure(foreground="#ffffff")
        self.Label1_2.configure(highlightbackground="#d9d9d9")
        self.Label1_2.configure(highlightcolor="black")
        self.Label1_2.configure(text='''Stats''')

        self.Label1_2 = tk.Label(top)
        self.Label1_2.place(relx=0.464, rely=0.679, height=46, width=111)
        self.Label1_2.configure(activebackground="#f9f9f9")
        self.Label1_2.configure(activeforeground="black")
        self.Label1_2.configure(background="#64ccc7")
        self.Label1_2.configure(disabledforeground="#a3a3a3")
        self.Label1_2.configure(font="-family {MS PGothic} -size 14 -weight bold")
        self.Label1_2.configure(foreground="#ffffff")
        self.Label1_2.configure(highlightbackground="#d9d9d9")
        self.Label1_2.configure(highlightcolor="black")
        self.Label1_2.configure(text='''Select stat''')

        top_list = ['Top ingredients Overall', 'Top 10 Breakfast ingredients', 'Top 10 Lunch ingredients',
                    'Top 10 Dinner ingredients']
        self.TCombobox2 = ttk.Combobox(top, values=top_list, state='readonly')
        self.TCombobox2.place(relx=0.411, rely=0.729, relheight=0.036
                              , relwidth=0.180)
        self.TCombobox2.configure(font="-family {Segoe UI} -size 14")
        self.TCombobox2.configure(textvariable=plot_support.combobox1)
        self.TCombobox2.configure(takefocus="")

        self.Button1_4 = tk.Button(top)
        self.Button1_4.place(relx=0.458, rely=0.789, height=43, width=138)
        self.Button1_4.configure(activebackground="#ececec")
        self.Button1_4.configure(activeforeground="#000000")
        self.Button1_4.configure(background="#1d9194")
        self.Button1_4.configure(disabledforeground="#a3a3a3")
        self.Button1_4.configure(font="-family {Segoe UI} -size 14 -weight bold")
        self.Button1_4.configure(foreground="#ffffff")
        self.Button1_4.configure(highlightbackground="#d9d9d9")
        self.Button1_4.configure(highlightcolor="black")
        self.Button1_4.configure(pady="0")
        self.Button1_4.configure(text='''Show stat''')
        self.Button1_4.configure(command=self.display_stat)

        self.top_text = tk.Label(top)
        self.top_text.place(relx=0.104, rely=0.889, height=46, width=391)
        self.top_text.configure(activebackground="#f9f9f9")
        self.top_text.configure(activeforeground="black")
        self.top_text.configure(background="#64ccc7")
        self.top_text.configure(disabledforeground="#a3a3a3")
        self.top_text.configure(font=font14)
        self.top_text.configure(foreground="#ffffff")
        self.top_text.configure(highlightbackground="#d9d9d9")
        self.top_text.configure(highlightcolor="black")

        self.Button2 = tk.Button(top)
        self.Button2.place(relx=0.932, rely=0.03, height=53, width=68)
        self.Button2.configure(activebackground="#ececec")
        self.Button2.configure(activeforeground="#000000")
        self.Button2.configure(background="#ea0f46")
        self.Button2.configure(disabledforeground="#a3a3a3")
        self.Button2.configure(font=font10)
        self.Button2.configure(foreground="#ffffff")
        self.Button2.configure(highlightbackground="#d9d9d9")
        self.Button2.configure(highlightcolor="black")
        self.Button2.configure(pady="0")
        self.Button2.configure(text='''X''')
        self.Button2.configure(command=root.destroy)

        self.top_text_4 = tk.Label(top)
        self.top_text_4.place(relx=0.318, rely=0.889, height=46, width=1230)
        self.top_text_4.configure(activebackground="#f9f9f9")
        self.top_text_4.configure(activeforeground="black")
        self.top_text_4.configure(background="#64ccc7")
        self.top_text_4.configure(disabledforeground="#a3a3a3")
        self.top_text_4.configure(font=font15)
        self.top_text_4.configure(foreground="#ffffff")
        self.top_text_4.configure(highlightbackground="#d9d9d9")
        self.top_text_4.configure(highlightcolor="black")


**Main method to invoke the GUI screen**

In [ ]:
if __name__ == '__main__':
    vp_start_gui()